In [3]:
import ovmsclient
import cv2
import torch
import numpy as np


In [4]:
# ! docker run -v $(pwd)/weights:/opt/weights -p 9000:9000 openvino/model_server:latest --model_path /opt/weights/yolov7 --model_name yolov7 --port 9000

In [5]:
from ovmsclient import make_grpc_client

client = make_grpc_client("localhost:9000")
client

In [6]:
from utils.datasets import letterbox
def process_image(image_path,stride ):
    image0 = cv2.imread(image_path)
    image0.shape

    image = letterbox(image0, 320, stride=stride)[0]
    image = image[:, :, ::-1].transpose(2, 0, 1)  # BGR to RGB, to 3x416x416
    image = np.ascontiguousarray(image)
    image = torch.from_numpy(image).to(device).half()
    image /= 255.0  # 0 - 255 to 0.0 - 1.0
    image = image.unsqueeze(0)
    image = image.detach().cpu().numpy()
    image = image.astype(np.float32)
    

    return image, image0

image_path = "/home/nguyen.thanh.huyenb/yolov7/training_minicoco_dataset/test/000000001722.jpg"
image_path = "/home/nguyen.thanh.huyenb/yolov7/training_minicoco_dataset/test/000000000086.jpg"
image_path = "/home/nguyen.thanh.huyenb/yolov7/training_minicoco_dataset/test/000000000247.jpg"
image_path = "/home/nguyen.thanh.huyenb/yolov7/training_minicoco_dataset/test/000000000471.jpg"
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
device
image, image0 = process_image(image_path, stride=32)
image.shape
image.dtype

dtype('float32')

In [7]:
output = client.predict({"images": image}, model_name="yolov7")

In [9]:
output["outputs"].shape

(1, 4410, 13)

In [11]:
from utils.general import scale_coords, non_max_suppression
def inference(preds, image, image0):
    names = ['bicycle', 'car', 'motorcycle', 'airplane', 'bus', 'train', 'truck', 'boat']
    preds = torch.Tensor(preds)
    preds = non_max_suppression(preds) # list
  
    pred = preds[0]
    print(pred.shape)
    pred[:, :4] = scale_coords(image.shape[2:], pred[:, :4], image0.shape).round()

    for *xyxy, conf, cls in reversed(pred):
        # print(xyxy)
        label = f'{names[int(cls)]} {conf:.2f} '
        print(label)

    return pred

preds = inference(output["outputs"], image, image0)

torch.Size([1, 6])
bus 0.96 


In [13]:
def visualize(image, pred, filename="/home/nguyen.thanh.huyenb/yolov7/visualize/test_onnx.jpg"):
    names = ['bicycle', 'car', 'motorcycle', 'airplane', 'bus', 'train', 'truck', 'boat']
    bboxes = []
    cls = []

    for p in pred:

        bboxes.append(p[:4])
        cls.append(p[5].item())
       
    
    for bbox, c in zip(bboxes, cls):
        x0, y0, x1, y1 = bbox[0], bbox[1],bbox[2], bbox[3]
       
        label = names[int(c)]
            
        x0, y0, x1, y1 = int(x0), int(y0), int(x1), int(y1)
        image = cv2.rectangle(image, (int(x0), int(y0)), (int(x1), int(y1)), (255, 0, 0), 1)
        image = cv2.putText(image, label, (x0 - 10, y0 - 10), cv2.FONT_HERSHEY_SIMPLEX,
                0.5, (255, 0, 0), 1, cv2.LINE_AA)


    cv2.imwrite(filename, image)
    print("Save results in ", filename)
        
visualize(image0, preds)

Save results in  /home/nguyen.thanh.huyenb/yolov7/visualize/test_onnx.jpg


: 